In [66]:
import pandas as pd
import numpy as np

import scipy.optimize as opt

from pathlib import Path
root = Path("../input/google-smartphone-decimeter-challenge/")

In [67]:
def ecef2lla(x, y, z):
    # x, y and z are scalars or vectors in meters
    x = np.array([x]).reshape(np.array([x]).shape[-1], 1)
    y = np.array([y]).reshape(np.array([y]).shape[-1], 1)
    z = np.array([z]).reshape(np.array([z]).shape[-1], 1)

    a=6378137
    a_sq=a**2
    e = 8.181919084261345e-2
    e_sq = 6.69437999014e-3

    f = 1/298.257223563
    b = a*(1-f)

    # calculations:
    r = np.sqrt(x**2 + y**2)
    ep_sq  = (a**2-b**2)/b**2
    ee = (a**2-b**2)
    f = (54*b**2)*(z**2)
    g = r**2 + (1 - e_sq)*(z**2) - e_sq*ee*2
    c = (e_sq**2)*f*r**2/(g**3)
    s = (1 + c + np.sqrt(c**2 + 2*c))**(1/3.)
    p = f/(3.*(g**2)*(s + (1./s) + 1)**2)
    q = np.sqrt(1 + 2*p*e_sq**2)
    r_0 = -(p*e_sq*r)/(1+q) + np.sqrt(0.5*(a**2)*(1+(1./q)) - p*(z**2)*(1-e_sq)/(q*(1+q)) - 0.5*p*(r**2))
    u = np.sqrt((r - e_sq*r_0)**2 + z**2)
    v = np.sqrt((r - e_sq*r_0)**2 + (1 - e_sq)*z**2)
    z_0 = (b**2)*z/(a*v)
    h = u*(1 - b**2/(a*v))
    phi = np.arctan((z + ep_sq*z_0)/r)
    lambd = np.arctan2(y, x)

    return phi*180/np.pi, lambd*180/np.pi, h

def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
      np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

# Apply WLS on one collection and one measurement

In [68]:
collection_name="2020-05-29-US-MTV-1"
file_path = Path(f"train/{collection_name}")
phone = 'Pixel4'
measurement_epoch_time = 1274827487438

# baseline we'll compare our solution against
df_baseline = pd.read_csv(root/"baseline_locations_train.csv")

# ground truth to compute methods performance
df_groundtruth = pd.read_csv(root/file_path/f"{phone}/ground_truth.csv")

# Train df here only contains one collection and one measurement
df_train = pd.read_csv(root/file_path/f"{phone}/{phone}_derived.csv")
df_train = df_train[df_train['millisSinceGpsEpoch'] == measurement_epoch_time] 

In [69]:
# Corrected pseudorange according to data instructions
df_train['correctedPrM'] = df_train.apply(
    lambda r: r.rawPrM + r.satClkBiasM - r.isrbM - r.ionoDelayM - r.tropoDelayM,
    axis=1
)

# Time it took for signal to travel
light_speed = 299_792_458
df_train['transmissionTimeSeconds'] = df_train['correctedPrM'] / light_speed

In [70]:
# Compute true sat positions at arrival time
omega_e = 7.2921151467e-5
df_train['xSatPosMRotated'] = \
    np.cos(omega_e * df_train['transmissionTimeSeconds']) * df_train['xSatPosM'] \
    + np.sin(omega_e * df_train['transmissionTimeSeconds']) * df_train['ySatPosM']
    
df_train['ySatPosMRotated'] = \
    - np.sin(omega_e * df_train['transmissionTimeSeconds']) * df_train['xSatPosM'] \
    + np.cos(omega_e * df_train['transmissionTimeSeconds']) * df_train['ySatPosM']
    
df_train['zSatPosMRotated'] = df_train['zSatPosM']

In [71]:
# Uncertainty weight for the WLS method
df_train['uncertaintyWeight'] = 1 / df_train['rawPrUncM']

In [72]:
df_train = df_train.iloc[0:10,:]

In [73]:
# Set up least squares methods
def distance(sat_pos, x):
    sat_pos_diff = sat_pos.copy(deep=True)
    
    sat_pos_diff['xSatPosMRotated'] = sat_pos_diff['xSatPosMRotated'] - x[0]
    sat_pos_diff['ySatPosMRotated'] = sat_pos_diff['ySatPosMRotated'] - x[1]
    sat_pos_diff['zSatPosMRotated'] = sat_pos_diff['zSatPosMRotated'] - x[2]

    sat_pos_diff['d'] = sat_pos_diff.apply(
        lambda r: r.uncertaintyWeight * 
            (np.sqrt((r.xSatPosMRotated**2 + r.ySatPosMRotated**2 + r.zSatPosMRotated**2)) + x[3] - r.correctedPrM),
        axis=1
    )

    return sat_pos_diff['d']

def distance_fixed_satpos(x):
    return distance(df_train[['xSatPosMRotated', 'ySatPosMRotated', 'zSatPosMRotated', 'correctedPrM', 'uncertaintyWeight']], x)

In [74]:
# Start point for the optimiser
x0= [0,0,0,0]

opt_res = opt.least_squares(distance_fixed_satpos, x0)

# Optimiser yields a position in the ECEF coordinates
opt_res_pos = opt_res.x

In [75]:
# ECEF position to lat/long
wls_estimated_pos = ecef2lla(*opt_res_pos[:3]) # x,y,z
wls_estimated_pos = np.squeeze(wls_estimated_pos) # １次元を削除

In [76]:
val_baseline = df_baseline[
    (df_baseline['collectionName']==collection_name)
    & (df_baseline['phoneName']==phone)
    & (df_baseline['millisSinceGpsEpoch']==measurement_epoch_time)
].iloc[0]

In [77]:
val_groundtruth = df_groundtruth[
    (df_groundtruth['collectionName']==collection_name)
    & (df_groundtruth['phoneName']==phone)
    & (df_groundtruth['millisSinceGpsEpoch']==measurement_epoch_time)
].iloc[0]

In [78]:
print("Baseline distance with groundtruth position (m)")
calc_haversine(val_baseline['latDeg'], val_baseline['lngDeg'], val_groundtruth['latDeg'], val_groundtruth['lngDeg'])

Baseline distance with groundtruth position (m)


15.419474998287948

In [79]:
print("Our estimated position (with WLS) distance with groundtruth position (m)")
calc_haversine(wls_estimated_pos[0], wls_estimated_pos[1], val_groundtruth['latDeg'], val_groundtruth['lngDeg'])

Our estimated position (with WLS) distance with groundtruth position (m)


12.820946086302946